In [ ]:
import os


os.getcwd()

DIR_PROJECT = os.path.dirname(os.getcwd())
VIDEO_PATH = os.path.join(DIR_PROJECT,"videos")
os.makedirs(VIDEO_PATH, exist_ok=True)


In [ ]:
import gpxpy

GPX_PATH = os.path.join(DIR_PROJECT,'gps','sample.gpx')

with open(GPX_PATH, "r") as f:
    print("Open gps file")
    gpx = gpxpy.parse(f)
    print("gps file opened")
    

print("Tracks:", len(gpx.tracks))